# 🧠 Brain MRI İkili Sınıflandırma — Modeli Tek Tek Seçerek Eğitim


Bu defter, **sadece ikili sınıflandırma** (labels: **`tumor`** ve **`no_tumor`**) için düzenlendi.
Aşağıdaki hücreler ile **modeli tek tek kendiniz seçip** eğitebilir; her eğitim için **ayrı ayrı**:
- Accuracy grafiği,
- Loss grafiği,
- **Hata matrisi (Confusion Matrix)**,
- **ROC eğrisi (AUC ile)**,
- ve **tablo halinde Sensitivity (Recall), Precision, F1, Cohen’s Kappa**

oluşturup **PNG olarak kaydedebilirsiniz**.

## 📦 Kurulumlar ve Kütüphaneler

In [1]:
!pip install -q timm scikit-learn torchmetrics

import os, math, time, random, copy
from dataclasses import dataclass
from typing import Dict, Any, Tuple, List

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms

import timm  # çok sayıda SOTA mimari
from sklearn.metrics import (
    confusion_matrix, roc_curve, auc,
    precision_score, recall_score, f1_score, cohen_kappa_score
)
import matplotlib.pyplot as plt

# Reprodüksiyon
def set_seed(seed: int = 42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# DataLoader işçileri için deterministik davranış
def seed_worker(worker_id: int):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

set_seed(42)

# (Opsiyonel) PyTorch 2.x’te matmul hesaplarını “high” hassasiyete çekerek olası hız optimizasyonu deniyor; desteklenmezse sessizce geçiyor.
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_WORKERS = min(4, os.cpu_count() or 1) # DataLoader için en fazla 4 olmak üzere CPU çekirdek sayısı kadar işçi belirliyor.
BATCH_SIZE = 32
VAL_RATIO = 0.1   # verinin %10’u validasyon için ayrılacak.
MAX_EPOCHS_DEFAULT = 50 
AUG_STRENGTH = 0.3  # 0.0 => aug kapalı, 0.3 => agresif (train tarafında)
print("Device:", DEVICE)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Device: cuda


## 🧭 Yollar ve Temel Ayarlar

In [2]:

# 🔧 DİZİN YAPISI
# DATA_ROOT:
#   ├── Training
#   │    ├── tumor
#   │    └── no_tumor
#   └── Testing
#        ├── tumor
#        └── no_tumor

DATA_ROOT = "Dataset"  # Veri seti kök dizin.
TRAIN_DIR = os.path.join(DATA_ROOT, "Training")
TEST_DIR  = os.path.join(DATA_ROOT, "Testing")

# Etiket sırası sabitliyorum: 0 = no_tumor, 1 = tumor
CLASS_NAMES = ["no_tumor", "tumor"]
POS_LABEL = 1  # ROC/metrics için pozitif sınıf (tumor)

os.makedirs("results", exist_ok=True) # Çıktılar için results klasörü yoksa oluşturuluyor.


## 🧩 Model Profilleri (Input Size) ve Tekli Seçim

In [3]:
# Giriş boyutu önerileri
MODEL_PROFILES = {
    "resnet34": {"input_size": 224},
    "resnet50": {"input_size": 224},
    "densenet121": {"input_size": 224},
    "inception_v3": {"input_size": 299},
    "efficientnet_b0": {"input_size": 224},
    "mobilenetv2_100": {"input_size": 224},
    "convnext_tiny": {"input_size": 224},

    # ✅ HYBRID
    "hybrid_dn121_effb0": {"input_size": 256},
}

ALL_MODELS = list(MODEL_PROFILES.keys())

# ✅ Senin kararın: dataset karışık olduğu için her şeyi 256'ya sabitle
OVERRIDE_INPUT_SIZE = 256

# Buradan TEK bir modeli seçin
SELECT_ONE_MODEL = "hybrid_dn121_effb0"  # <- hibrit modeli seçtik
TARGET_MODEL = SELECT_ONE_MODEL

INPUT_SIZE = OVERRIDE_INPUT_SIZE if OVERRIDE_INPUT_SIZE is not None else MODEL_PROFILES[SELECT_ONE_MODEL]["input_size"]
print("Selected:", SELECT_ONE_MODEL, "Input:", INPUT_SIZE)


Selected: hybrid_dn121_effb0 Input: 256


## 🖼️ Dönüşümler (Pad + Resize + Normalize)

In [4]:
from timm.data import resolve_data_config

# Model bazlı mean/std cache
_MEAN_STD_CACHE = {}

# ✅ Hibrit için mean/std hangi backbone'dan alınacak?
_HYBRID_MEANSTD_SOURCE = {
    "hybrid_dn121_effb0": "densenet121"  # ikisi de ImageNet normalize kullandığı için fark etmez
}

def get_mean_std_from_timm(model_name: str):
    """
    timm modelinin default_cfg/pretrained_cfg içinden mean/std çeker.
    Hibrit model adını desteklemek için backbone'a map edilir.
    """
    name = _HYBRID_MEANSTD_SOURCE.get(model_name, model_name)

    if name in _MEAN_STD_CACHE:
        return _MEAN_STD_CACHE[name]

    m = timm.create_model(name, pretrained=False, num_classes=2)
    cfg = resolve_data_config({}, model=m)

    mean, std = cfg["mean"], cfg["std"]
    _MEAN_STD_CACHE[name] = (mean, std)
    del m
    return mean, std


class SquarePad:
    def __call__(self, img):
        w, h = img.size
        if w == h:
            return img
        size = max(w, h)
        pad_left = (size - w) // 2
        pad_top = (size - h) // 2
        pad_right = size - w - pad_left
        pad_bottom = size - h - pad_top
        return transforms.functional.pad(
            img, (pad_left, pad_top, pad_right, pad_bottom), fill=0
        )


def build_transforms(model_name: str, input_size: int, aug_strength: float = 0.0):
    """
    Transform üretimi.

    aug_strength:
      - 0.0  => augmentasyon kapalı (train = eval)
      - 0.0 < => train'de augmentasyon açık (0.3 = agresif)
    Not: aug_strength 0.3 üstü verilirse 0.3'e kırpılır.
    """
    mean, std = get_mean_std_from_timm(model_name)

    base = transforms.Compose([
        SquarePad(),
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ])

    # Aug kapalıysa train = eval (tam istediğin davranış)
    try:
        a = float(aug_strength)
    except Exception:
        a = 0.0
    a = max(0.0, min(0.3, a))
    if a <= 0.0:
        return base, base

    # 0..1 ölçeğe taşı (0.3 => 1.0 yoğunluk)
    s = a / 0.3

    # PIL seviyesinde (ToTensor'dan önce) augmentasyonlar
    pil_aug = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.50 * s),
        transforms.RandomVerticalFlip(p=0.15 * s),
        transforms.RandomRotation(degrees=15.0 * s),
        transforms.RandomAffine(
            degrees=0.0,
            translate=(0.08 * s, 0.08 * s),
            scale=(1.0 - 0.10 * s, 1.0 + 0.10 * s),
        ),
        transforms.RandomPerspective(distortion_scale=0.20 * s, p=0.25 * s),
        transforms.RandomApply(
            [transforms.ColorJitter(
                brightness=0.40 * s,
                contrast=0.40 * s,
                saturation=0.25 * s,
                hue=0.05 * s,
            )],
            p=0.70 * s
        ),
        transforms.RandomApply(
            [transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.5 * s + 0.1))],
            p=0.20 * s
        ),
    ])

    train_tfms = transforms.Compose([
        SquarePad(),
        transforms.Resize((input_size, input_size)),
        pil_aug,
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
        # Tensor seviyesinde augmentasyon (masking benzeri regularization)
        transforms.RandomErasing(
            p=0.25 * s,
            scale=(0.02, 0.10),
            ratio=(0.3, 3.3),
            value="random"
        ),
    ])

    eval_tfms = base
    return train_tfms, eval_tfms


## 📥 Dataset & DataLoader'lar

In [5]:
import os
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets
from sklearn.model_selection import StratifiedShuffleSplit

def make_dataloaders(model_name: str, input_size: int, batch_size: int = BATCH_SIZE, val_ratio: float = VAL_RATIO, aug_strength: float = 0.0):
    """
    Train/Val/test DataLoader kurulumunu doğru transform'larla yapar.
    - Train: train_tfms
    - Val/Test: eval_tfms
    Ayrıca train subset'i geri döndürür (sınıf ağırlıkları için).
    """
    train_tfms, eval_tfms = build_transforms(model_name, input_size, aug_strength=aug_strength)

    # 1) Stratified split için base dataset + etiketler
    base_ds = datasets.ImageFolder(TRAIN_DIR)  # transform YOK
    y = np.array(base_ds.targets)
    sss = StratifiedShuffleSplit(n_splits=1, test_size=val_ratio, random_state=42)
    train_indices, val_indices = next(sss.split(np.zeros(len(y)), y))

    # 2) Ayrı görünümler: train vs val/test için farklı transform
    train_view = datasets.ImageFolder(TRAIN_DIR, transform=train_tfms)
    val_view   = datasets.ImageFolder(TRAIN_DIR, transform=eval_tfms)
    test_ds    = datasets.ImageFolder(TEST_DIR,  transform=eval_tfms)

    # 3) İndeksleri Subset'lere uygula
    train_ds = Subset(train_view, train_indices.tolist())
    val_ds   = Subset(val_view,   val_indices.tolist())

    # 4) (Opsiyonel) Sınıf sırası kontrolü
    expected = ["no_tumor", "tumor"]
    assert train_view.classes == expected, f"Sınıf sırası {train_view.classes} beklenen {expected} değil!"

    # 5) DataLoader'lar (deterministik workers)
    gen = torch.Generator().manual_seed(42)
    pin = (DEVICE.type == "cuda")
    common = dict(num_workers=NUM_WORKERS, pin_memory=pin, worker_init_fn=seed_worker, persistent_workers=bool(NUM_WORKERS))

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  generator=gen, **common)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, generator=gen, **common)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, generator=gen, **common)

    return train_loader, val_loader, test_loader, train_ds


## 🏗️ Model & Kayıp Fonksiyonu & Optimizasyon

In [6]:
class HybridDN121_EffB0(nn.Module):
    """
    DenseNet121 + EfficientNetB0 hibrit:
    - İki backbone'dan pooled feature çıkarır (num_classes=0, global_pool='avg')
    - Concatenate + küçük bir MLP head ile 2 sınıf logits üretir
    """
    def __init__(self, num_classes=2, head_dim=256, dropout=0.2, freeze_backbones=False):
        super().__init__()
        self.bb1 = timm.create_model("densenet121", pretrained=True, num_classes=0, global_pool="avg")
        self.bb2 = timm.create_model("efficientnet_b0", pretrained=True, num_classes=0, global_pool="avg")

        if freeze_backbones:
            for p in self.bb1.parameters(): p.requires_grad = False
            for p in self.bb2.parameters(): p.requires_grad = False

        dim1 = getattr(self.bb1, "num_features")
        dim2 = getattr(self.bb2, "num_features")

        self.head = nn.Sequential(
            nn.Linear(dim1 + dim2, head_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(head_dim, num_classes)   # ✅ logits (softmax yok)
        )

    def forward(self, x):
        f1 = self.bb1(x)
        f2 = self.bb2(x)
        feats = torch.cat([f1, f2], dim=1)
        return self.head(feats)


def build_model(model_name: str, num_classes: int = 2):
    if model_name == "hybrid_dn121_effb0":
        # freeze_backbones=True dersen overfitting azalır ama tavan performansı düşebilir
        return HybridDN121_EffB0(num_classes=num_classes, head_dim=256, dropout=0.2, freeze_backbones=False)

    # normal timm modeller
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
    return model


from torch.utils.data import Subset
def compute_class_weights(dataset):
    """
    ImageFolder ya da Subset(ImageFolder) kabul eder.
    """
    # Subset ise hedef etiketleri indekslerden topla
    if isinstance(dataset, Subset):
        base = dataset.dataset
        indices = dataset.indices
        targets = getattr(base, "targets", None)
        if targets is None:
            raise ValueError("Temel dataset'te 'targets' bulunamadı.")
        labels = [targets[i] for i in indices]
    else:
        labels = list(getattr(dataset, "targets", []))

    n0 = sum(1 for y in labels if y == 0)
    n1 = sum(1 for y in labels if y == 1)
    total = max(1, n0 + n1)
    # Basit ters frekans ağırlıklandırması
    w0 = total / (2.0 * max(1, n0))
    w1 = total / (2.0 * max(1, n1))
    return torch.tensor([w0, w1], dtype=torch.float)
    
def make_optimizer(model, lr=3e-4, weight_decay=1e-4):
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    return optimizer


## 🔁 Eğitim & Doğrulama Döngüsü (History ile)

In [7]:
@dataclass
class History:
    train_loss: list
    val_loss: list
    train_acc: list
    val_acc: list

from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

def run_one_epoch(model, loader, criterion, optimizer=None, scaler: GradScaler = None):
    is_train = optimizer is not None
    model.train(mode=is_train)

    total_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images = images.to(DEVICE, non_blocking=True)
        labels = labels.to(DEVICE, non_blocking=True)

        with autocast(enabled=(DEVICE.type == "cuda")):
            outputs = model(images)
            loss = criterion(outputs, labels)

        if is_train:
            optimizer.zero_grad(set_to_none=True)
            if scaler is not None and DEVICE.type == "cuda":
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

        total_loss += loss.item() * labels.size(0)
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / max(1, total)
    acc = correct / max(1, total)
    return avg_loss, acc

def train_model_one_run(model_name: str,
                        max_epochs: int = MAX_EPOCHS_DEFAULT,
                        lr: float = 3e-4,
                        weight_decay: float = 1e-4,
                        batch_size: int = BATCH_SIZE,
                        aug_strength: float = 0.0):
    input_size = MODEL_PROFILES[model_name]["input_size"]

    # ÖNEMLİ: Artık batch_size parametresini gerçekten kullanıyoruz
    train_loader, val_loader, test_loader, full_train = make_dataloaders(
        model_name=model_name,
        input_size=input_size,
        batch_size=batch_size,
        aug_strength=aug_strength
    )


    model = build_model(model_name).to(DEVICE)
    class_weights = compute_class_weights(full_train).to(DEVICE)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = make_optimizer(model, lr=lr, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3)
    scaler = GradScaler(enabled=(DEVICE.type == "cuda"))

    history = History(train_loss=[], val_loss=[], train_acc=[], val_acc=[])

    best_val = float("inf")
    best_state = None
    patience = 7
    patience_ctr = 0

    for epoch in range(max_epochs):
        tr_loss, tr_acc = run_one_epoch(
            model, train_loader, criterion,
            optimizer=optimizer, scaler=scaler
        )
        va_loss, va_acc = run_one_epoch(
            model, val_loader, criterion,
            optimizer=None, scaler=None
        )

        history.train_loss.append(tr_loss)
        history.val_loss.append(va_loss)
        history.train_acc.append(tr_acc)
        history.val_acc.append(va_acc)

        scheduler.step(va_loss)

        if va_loss < best_val:
            best_val = va_loss
            best_state = copy.deepcopy(model.state_dict())
            patience_ctr = 0
        else:
            patience_ctr += 1
            if patience_ctr >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    return model, history, val_loader, test_loader

# 🔎 Random Search Yardımcıları

In [8]:
# =========================
# 🔎 Random Search Yardımcıları
# =========================
import math, json, time, random
import numpy as np
import pandas as pd
from pathlib import Path

def sample_from(space: dict, rng: random.Random) -> dict:
    """
    Hyperparam alanından tek bir örnek seçer.
    Not: rng dışarıdan verilir; global random seed resetlerinden etkilenmez.
    
    Hyperparam alanından tek bir örnek seçer.
    space formatı örn:
    {
        "lr": {"type": "loguniform", "low": 1e-5, "high": 1e-2},
        "weight_decay": {"type": "loguniform", "low": 1e-6, "high": 1e-3},
        "batch_size": {"type": "choice", "values": [8, 16, 32]},
        "model_name": {"type": "choice", "values": ["efficientnet_v2_m", "resnet50"]}
    }
    """
    out = {}
    for k, cfg in space.items():
        t = cfg["type"]
        if t == "choice":
            out[k] = rng.choice(cfg["values"])
        elif t == "uniform":
            lo, hi = float(cfg["low"]), float(cfg["high"])
            out[k] = rng.random() * (hi - lo) + lo
        elif t == "loguniform":
            lo, hi = math.log(float(cfg["low"])), math.log(float(cfg["high"]))
            out[k] = math.exp(rng.random() * (hi - lo) + lo)
        else:
            raise ValueError(f"Bilinmeyen tür: {t}")
    return out


def ensure_dir(p):
    Path(p).mkdir(parents=True, exist_ok=True)

def try_set_seed(seed: int):
    try:
        set_seed(seed)  # Notebook'ta varsa kullan
    except Exception:
        # Yoksa sessizce geç
        pass

def run_one_trial(cfg,
                  max_epochs,
                  output_root,
                  model_name,
                  metric="val_acc",
                  seed=42,
                  aug_strength: float = 0.0):
    try_set_seed(seed)

    model, history,_, test_loader = train_model_one_run(
        model_name=model_name,
        max_epochs=max_epochs,
        lr=cfg["lr"],
        weight_decay=cfg["weight_decay"],
        batch_size=cfg["batch_size"],
        aug_strength=aug_strength
    )

    trial_name = (
        f"{model_name}_lr{cfg['lr']:.2e}"
        f"_wd{cfg['weight_decay']:.2e}"
        f"_bs{cfg['batch_size']}" f"_aug{aug_strength:.2f}"
    )
    trial_dir = output_root / trial_name
    ensure_dir(trial_dir)

    try:
        plot_and_save_history(history, trial_dir)
    except Exception as e:
        print("Plot failed:", e)

    # Metric seçimi
    if metric == "val_loss" and history.val_loss:
        metric_value = min(history.val_loss)
    elif metric == "val_acc" and history.val_acc:
        metric_value = max(history.val_acc)
    else:
        print(f"[WARN] Metric {metric} desteklenmiyor, val_acc kullanılacak.")
        metric_value = max(history.val_acc) if history.val_acc else float("-inf")

    return metric_value, trial_dir


def random_search(space,
                  n_trials,
                  max_epochs,
                  output_root,
                  metric="val_acc",
                  greater_is_better=True,
                  model_name=None,
                  aug_strength: float = 0.0):
    """
    Tek bir model (model_name) için random search yapar.
    space: lr, weight_decay, batch_size gibi hiperparametre aralığı
    """
    ensure_dir(output_root)
    output_root = Path(output_root)

    results = []
    sampler_rng = random.Random(12345)  # sadece hyperparam örneklemek için ayrı RNG

    for i in range(n_trials):
        # Hiperparametreleri random seç
        cfg = sample_from(space, sampler_rng)

        # Bir trial koş
        metric_value, trial_dir = run_one_trial(
            cfg=cfg,
            max_epochs=max_epochs,
            output_root=output_root,
            model_name=model_name,
            metric=metric,
            seed=42 + i,
            aug_strength=aug_strength
        )

        # Sonucu kaydet
        row = {
            **cfg,
            "metric_name": metric,
            "metric_value": metric_value,
            "trial_dir": str(trial_dir),
        }
        results.append(row)

    # En iyi sonucu seç
    if greater_is_better:
        best_row = max(results, key=lambda r: r["metric_value"])
    else:
        best_row = min(results, key=lambda r: r["metric_value"])

    best_cfg = {k: best_row[k] for k in space.keys()}

    # Pandas DataFrame'e çevir
    import pandas as pd
    df = pd.DataFrame(results)

    # CSV olarak kaydet
    csv_path = output_root / "random_search_results.csv"
    df.to_csv(csv_path, index=False)
    print("Random search sonuçları kaydedildi:", csv_path)

    return best_cfg, df



# 🚀 Random Search'i Çalıştır

In [ ]:
# =========================
# 🚀 Random Search'i Çalıştır
# =========================

# 1) Arama alanını tanımla (artık model_name yok, sadece hiperparametreler var)
search_space = {
    "lr": {
        "type": "loguniform",
        "low": 1e-5,
        "high": 3e-3,
    },
    "weight_decay": {
        "type": "loguniform",
        "low": 1e-6,
        "high": 1e-3,
    },
    "batch_size": {
        "type": "choice",
        "values": [8, 16, 24, 32],
    },
}

# 2) Süpürme ayarları
N_TRIALS   = 10          # Kaç deneme yapılacak
MAX_EPOCHS = 8           # Her denemenin epoch sayısı
METRIC     = "val_acc"   # Artık AUC değil, validation accuracy kullanıyoruz
HIGHER_BETTER = True     # val_acc için büyük olan daha iyidir
OUTPUT_ROOT = "./sweeps/random_search"

# 3) Çalıştır
best, df = random_search(
    space=search_space,
    n_trials=N_TRIALS,
    max_epochs=MAX_EPOCHS,
    output_root=OUTPUT_ROOT,
    metric=METRIC,
    greater_is_better=HIGHER_BETTER,
    model_name=TARGET_MODEL,   # >>> BURASI ÖNEMLİ: sadece seçtiğin modeli geziyoruz
    aug_strength=AUG_STRENGTH,
)

print("\nSeçilen model:", TARGET_MODEL)
print("En iyi konfigürasyon:")
print(best)

# DataFrame'i görüntüle
try:
    import IPython
    from IPython.display import display
    display(df.sort_values("metric_value", ascending=not HIGHER_BETTER))
except Exception:
    pass


## 📊 Değerlendirme + Grafik ve Tablo Kaydı

In [10]:
def ensure_dir(p):
    os.makedirs(p, exist_ok=True)

def plot_and_save_history(hist, out_dir: str):
    # Accuracy
    plt.figure()
    plt.plot(range(1, len(hist.train_acc)+1), hist.train_acc, label="train_acc")
    plt.plot(range(1, len(hist.val_acc)+1),   hist.val_acc,   label="val_acc")
    plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.title("Accuracy")
    plt.legend(); plt.grid(True, linestyle="--", alpha=0.4)
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "accuracy.png"), dpi=200)
    plt.close()

    # Loss
    plt.figure()
    plt.plot(range(1, len(hist.train_loss)+1), hist.train_loss, label="train_loss")
    plt.plot(range(1, len(hist.val_loss)+1),   hist.val_loss,   label="val_loss")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Loss")
    plt.legend(); plt.grid(True, linestyle="--", alpha=0.4)
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "loss.png"), dpi=200)
    plt.close()

def evaluate_and_save(model, test_loader, out_dir: str, model_name: str, threshold: float = 0.5):

    model.eval()
    y_true, y_prob, y_pred = [], [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(DEVICE, non_blocking=True)
            logits = model(images)

            probs = torch.softmax(logits, dim=1)[:, 1]          # tensor
            preds = (probs >= threshold).long()                 # ✅ threshold ile karar

            probs = probs.cpu().numpy()
            preds = preds.cpu().numpy()

            y_true.extend(labels.numpy().tolist())     # istersen alttaki "güvenli" versiyona geç
            y_prob.extend(probs.tolist())              # ✅ probs zaten numpy
            y_pred.extend(preds.tolist())              # ✅ preds zaten numpy


    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_prob = np.array(y_prob)

    # === Hata Matrisi ===
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    fig = plt.figure()
    plt.imshow(cm, interpolation='nearest')
    plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, CLASS_NAMES, rotation=45)
    plt.yticks(tick_marks, CLASS_NAMES)
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "confusion_matrix.png"), dpi=200)
    plt.close(fig)

    # === ROC Eğrisi ===
    from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report
    fpr, tpr, _ = roc_curve(y_true, y_prob, pos_label=1)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.4f})")
    plt.plot([0,1], [0,1], linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Sensitivity)")
    plt.title("ROC Curve")
    plt.legend(loc="lower right")
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "roc_curve.png"), dpi=200)
    plt.close()

    # === Metrikler ===
    precision = precision_score(y_true, y_pred, pos_label=1, zero_division=0)
    recall    = recall_score(y_true, y_pred,    pos_label=1, zero_division=0)  # sensitivity
    f1        = f1_score(y_true, y_pred,        pos_label=1, zero_division=0)
    kappa     = cohen_kappa_score(y_true, y_pred)
    acc       = accuracy_score(y_true, y_pred)

    # Tabloyu PNG olarak kaydet
    fig, ax = plt.subplots()
    ax.axis('off')
    ax.axis('tight')
    cell_text = [[f"{precision:.4f}", f"{recall:.4f}", f"{f1:.4f}", f"{kappa:.4f}", f"{acc:.4f}"]]
    col_labels = ["Precision", "Sensitivity (Recall)", "F1-Score", "Cohen's Kappa", "Accuracy"]
    the_table = ax.table(cellText=cell_text, colLabels=col_labels, loc='center')
    the_table.auto_set_font_size(False)
    the_table.set_fontsize(11)
    the_table.scale(1.2, 1.6)
    plt.title("Değerlendirme Metrikleri")
    fig.tight_layout()
    fig.savefig(os.path.join(out_dir, "metrics_table.png"), dpi=200)
    plt.close(fig)

    # Ayrıca CSV ve classification_report da kaydedelim
    import csv
    with open(os.path.join(out_dir, "metrics.csv"), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["metric", "value"])
        writer.writerow(["precision", precision])
        writer.writerow(["recall_sensitivity", recall])
        writer.writerow(["f1", f1])
        writer.writerow(["kappa", kappa])
        writer.writerow(["auc", roc_auc])
        writer.writerow(["accuracy", acc])

    with open(os.path.join(out_dir, "classification_report.txt"), "w") as f:
        f.write(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

    return {
        "precision": precision,
        "recall_sensitivity": recall,
        "f1": f1,
        "kappa": kappa,
        "auc": roc_auc,
        "accuracy": acc,
        "confusion_matrix": cm.tolist(),
    }


In [11]:
import numpy as np
from sklearn.metrics import f1_score, balanced_accuracy_score

@torch.no_grad()
def collect_probs(model, loader, device):
    model.eval()
    y_true, y_prob = [], []

    for images, labels in loader:
        images = images.to(device, non_blocking=True)
        logits = model(images)

        probs = torch.softmax(logits, dim=1)[:, 1]  # tumor olasılığı
        y_true.append(labels.cpu().numpy())
        y_prob.append(probs.cpu().numpy())

    return np.concatenate(y_true), np.concatenate(y_prob)

def select_threshold_on_val(y_true, y_prob, objective="f1"):
    thresholds = np.linspace(0.0, 1.0, 1001)
    best_t, best_score = 0.5, -1.0

    for t in thresholds:
        y_pred = (y_prob >= t).astype(int)

        if objective == "f1":
            score = f1_score(y_true, y_pred, pos_label=1)
        elif objective == "bal":
            score = balanced_accuracy_score(y_true, y_pred)
        else:
            raise ValueError("objective must be 'f1' or 'bal'")

        if score > best_score:
            best_score = score
            best_t = float(t)

    return best_t, best_score


# 🏁 En iyi konfigürasyonla yeniden-eğitim

In [13]:
print("Random Search en iyi hiperparametreler:")

FINAL_EPOCHS = 50  # İstersen artır

# Reprodüksiyon için seed
try_set_seed(42)

# Eğitim
model, history, val_loader, test_loader = train_model_one_run(
    model_name=TARGET_MODEL,
    max_epochs=FINAL_EPOCHS,
    lr=6.934684230776975e-05,
    weight_decay=0.000798817143317339,
    batch_size=32,
    aug_strength=AUG_STRENGTH,
)

# Çıktıları kaydet
final_out_dir = os.path.join("results", f"{TARGET_MODEL}_final")
os.makedirs(final_out_dir, exist_ok=True)

torch.save(model.state_dict(), os.path.join(final_out_dir, "best.pt"))
plot_and_save_history(history, final_out_dir)

# ✅ 1) VAL'dan threshold seç (test'e dokunma)
y_val, p_val = collect_probs(model, val_loader, DEVICE)
best_thr, best_obj = select_threshold_on_val(y_val, p_val, objective="f1")
print("VAL'dan seçilen en iyi threshold:", best_thr, "obj_score:", best_obj)

with open(os.path.join(final_out_dir, "best_threshold.txt"), "w", encoding="utf-8") as f:
    f.write(f"best_thr={best_thr}\nobjective=f1\nobj_score={best_obj}\n")

# ✅ 2) TEST'i bu threshold ile değerlendir
summary = evaluate_and_save(model, test_loader, final_out_dir, TARGET_MODEL, threshold=best_thr)

print("\nFinal eğitim tamamlandı.")
print("Metrikler:", summary)
print("Sonuç klasörü:", final_out_dir)


Random Search en iyi hiperparametreler:


/tmp/ipykernel_339183/1300154623.py:65: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(DEVICE.type == "cuda"))
/tmp/ipykernel_339183/1300154623.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(DEVICE.type == "cuda")):


Early stopping at epoch 22
VAL'dan seçilen en iyi threshold: 0.023 obj_score: 1.0

Final eğitim tamamlandı.
Metrikler: {'precision': 1.0, 'recall_sensitivity': 0.7262839879154078, 'f1': 0.8414420721036052, 'kappa': 0.7227557339788279, 'auc': 0.9665261334976879, 'accuracy': 0.8607439286812173, 'confusion_matrix': [[1598, 0], [453, 1202]]}
Sonuç klasörü: results/hybrid_dn121_effb0_final
